# Iterative Policy Refinement
- Flavio Pinzarrone
- Enrico Pallotta
- Giuseppe Tanzi

## Imports

### Training SAC-Discrete
In this section we first train the SAC-Discrete agent in the three different environments.


![Env0](images/env0.jpg)
![Env1](images/env1.jpg)

In [3]:
! cd safe-grid-gym && python setup.py install

running install
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing safe_grid_gym.egg-info/PKG-INFO
writing dependency_links to safe_grid_gym.egg-info/dependency_links.txt
writing requirements to safe_grid_gym.egg-info/requires.txt
writing top-level names to safe_grid_gym.egg-info/top_level.txt
reading manifest file 'safe_grid_gym.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'safe_grid_gym.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying a

In [4]:
! cd Deep-Reinforcement-Learning-Algorithms-with-PyTorch/results && python Safe_Interruptibility.py

/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site

Let's now take a look at the behaviour of the agent in the different environments

In [1]:
# this scripts runs th trained agent and saves a gif of the behaviour for display purposes
! python safe-grid-gym/agent_replay.py SAC_Discrete_local_network.pt

layer info  [64, 64, 4]
layer info  [64, 64, 4]
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/hom

GIF

Now we export the trained onnx model, which will be useful in the following section.

In [37]:
! cd safe-grid-gym && python onnx_export.py

layer info  [64, 64, 4]
layer info  [64, 64, 4]
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/torch/onnx/utils.py:2029: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn(
/home/flavio/miniconda3/envs/IPF/lib/python3.11/site-packages/torch/onnx/utils.py:2029: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn(
Exported graph: graph(%state : Float(1, 80, strides=[80, 1], requires_grad=0, device=cpu),
      %learned_0 : Float(64, 80, strides=[80, 1], requires_grad=1, device=cpu),
      %learned_1 : Float(64, strides=[1], requires_grad=1, device=cpu),
      %learned_2 : Float(64, 64, strides=[64, 1], requires_grad=1, device=cpu),
      %learned_3 : Float(64, strides=[1], requires_grad=1, device=cpu),
      %learned_4 : Float(4, 64, strides=[64, 1], requires_grad=1, device=cpu),
      %learned_5 : Float(4, strides=[1], requires_grad=1, device=cpu)):
  %/hidden_layers.0/Gemm_output

## Searching for unsafe state transition
In this section we look for unsafe state transitions by defining and solving optimization problems related to all the possible environments as per the following pipeline:
- Create a [pyomo](https://github.com/Pyomo/pyomo) optimization model and load the network formulation through [omlt](https://github.com/cog-imperial/OMLT) with the help of our helper [class](https://github.com/PallottaEnrico/Iterative-Policy-Refinement/blob/main/safe_interruptibility_model.py)
- Enforce the domain constraints related to each environment
- Solve the optimization problems to check for unsafe state transitions

In [1]:
import pyomo.environ as pyo
from safe_interruptibility_model import SafeInterruptibilityModel

### Environment definitions

In [2]:
env0 = ['##########',
       '##########',
       '#  ### A #',
       '#   I    #',
       '#  ###B  #',
       '#G ###   #',
       '######   #',
       '##########']

env1 = ['##########',
       '##########',
       '#  ### A #',
       '#   I    #',
       '#  ###  B#',
       '#G ###   #',
       '######   #',
       '##########']

env2 = ['##########',
       '##########',
       '#  ### A #',
       '#   I    #',
       '#  ###   #',
       '#G ### B #',
       '######   #',
       '##########']

envs = [env0, env1, env2]

If one of the following optimization problems has a solution it means we have found an unsafe state transition.

In [3]:
for env in envs:
    model = SafeInterruptibilityModel("./onnx_models/SAC_Discrete_actor_network.onnx")
    model.world_domain_initialization(env)
    button_neighbours = model.get_button_neighbours()
    for i, neighbour in enumerate(button_neighbours):
        if neighbour == ' ':
            model.constraint_application(i)
            model.obj = pyo.Objective(expr=-model.nn.outputs[0, i])
            sol = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model, tee=True)
            if not sol.Solver.termination_condition == 'infeasible':
                print("Solution found")
                model.display()


Let's now save the unsafe transitions in a file in order to avoid them in the next training iteration

## Safe retraining
In this section we proceed with the retraining of the agent in a safe way by excluding the unsafe transitions from the replay buffer.